<a href="https://colab.research.google.com/github/wuy54/Instagram-picture-magnifier/blob/main/Assignment_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with data that was downloaded using using the [yfinance](https://pypi.org/project/yfinance/) library. It containst daily closing prices of all symbols.

In [1]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd
import numpy as np

In [2]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())


In [ ]:
#Describe your answer below.
#Answer 1: 
# The first line is to read the csv file, which is the linked .txt file as
# the first parameter of the function, with a delimiter "|" to separate 
# string values and put the values into the data frame variable "data".
# It uses read_csv function imported from "pandas" package.
#
# The second line is setting a data frame 'data_clean' with only data in the 
# data frame "data" that the "Test Issue" has a value of "N"

# The third line assigns a set of all possible values of 'NASDAQ Symbol' to 
# the variable "symbols"

# The forth line reads another csv file and overwrite all data saved in the 
# dataframe "data" with new ones.

# The fifth line is updating the data frame 'data_clean' with only data in the 
# data frame "data" that the "Test Issue" has a value of "N" but with the new
# "data" dataframe.

# The sixth line is doing the same thing as the third line but with the
# updated "data_clean"

In [3]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2023-01-26 11:30:17--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/gzpi950gfdz9dsk/stocks.parquet [following]
--2023-01-26 11:30:17--  https://www.dropbox.com/s/dl/gzpi950gfdz9dsk/stocks.parquet
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2bc54d47044af8f323e6e64f41.dl.dropboxusercontent.com/cd/0/get/B1RU8j3CljJijQU8C7Fbf4WwCzzii_pFcdl3Zi6Vr3kGe6FVCRTs87yg0voZGRn-nHRR_IFhcMga6mJi5-m9mlDIVZNujARDL7Nlj9v19OTMJyPf-uYzCzvasODiD1ZcLfVsEtNm-ik--aQbBmRrKkbT9FArPGIBkJC6kzmmeT-A3Q/file?dl=1# [following]
--2023-01-26 11:30:17--  https://uc2bc54d47044af8f323e6e64f41.dl.dropboxusercontent.com/cd/0/get/B1RU8j3CljJijQU8C7Fbf4WwCzzii_pFcdl3Zi6Vr3kGe6FVCRTs87yg0voZGRn-nHRR_IFh

In [4]:
!ls stocks.parquet -l #downloaded file

-rw-rw-r-- 1 ubuntu ubuntu 678640589 Jan 26 11:30 stocks.parquet


In [ ]:
#Bonus question 1 - find on internet and write the command that will download the all data from the symbols list. 

In [5]:
data=pd.read_parquet('stocks.parquet')

In [6]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock


,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday


In [42]:
# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the adj_close on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z
# Sort the stocks by this return and find the top 20

df = data[['symbol', 'date', 'adj_close']]
df = df.dropna()
df = df[df != 0]
df = df.sort_values(['date'])
df_first = df.drop_duplicates('symbol', keep='first')
df_last = df.drop_duplicates('symbol', keep='last')
df = pd.merge(df_first, df_last, on="symbol")
df['day_passed'] = (pd.to_datetime(df['date_y']) - \
                    pd.to_datetime(df['date_x'])).dt.days
df = df[df['day_passed'] != 0] # removing stocks with only one day of data
df['return'] = df['adj_close_y'] / df['adj_close_x'] / df['day_passed']
df = df.sort_values(by='return', ascending=False)
df_result = df.head(20)[['symbol', 'return']]
print(df_result)



      symbol        return
509      NVO  3.302876e+06
499      NKE  2.959418e+03
289      DCI  3.869564e+02
789      TJX  2.063167e+02
346     WTRG  7.221605e+00
422      TTC  6.224708e+00
158      USB  4.078555e+00
27       MCD  3.374097e+00
809      PII  2.554223e+00
516       HD  1.506773e+00
11029   IQMD  1.001014e+00
11032  AOGOU  9.990020e-01
11030   PCCT  9.959839e-01
34       ETN  9.076372e-01
11031  FRLAW  8.888889e-01
368      HAS  7.035260e-01
11        KO  6.680578e-01
10        MO  6.485381e-01
3457     KOL  5.786728e-01
432      HRL  5.406652e-01


In [ ]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS
# 
# Answer: Based on my attemps, the code runs slower on AWS than my local machine.